# Hyperparameter Tuning
### The process of chossing optimal parameter is called hyper tuning

### hyperparameter might address model design questions such as:
- 1 What degree of ponomial features should i use for my linear model?
- 2 What should be the maximum depth allowed for my decision tree?
- 3 What shpuld be the minimum number of samples required at a leaf node in my decision tree?
- 4 How many tree should I include in my random forest?
- 5 How many neuron should I have in my neural network layeer?
- 6 How many layers should I have in my neural network?
- 7 What should set my learning rateto for gradient descent?


In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [4]:
import pandas as pd
df= pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,0
48,5.3,3.7,1.5,0.2,0
49,5.0,3.3,1.4,0.2,0
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1


In [5]:
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [18]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(iris.data,iris.target,test_size =  0.3)


In [19]:
model = svm.SVC(kernel= 'rbf',C=30,gamma= 'auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9555555555555556

this score is not realiable as score is changes with the dataset again and again so here we use "K fold cross validation"


In [30]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [31]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [33]:
cross_val_score(svm.SVC(kernel='linear',C=20,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

we can take the mean and find out the optimal value for this parametes

but this method is very manual and repetative so another method is to run for loop for this


In [34]:
import numpy as np
kernels=['rbf','linear']
C=[1,10,20]
avg_scores={}
for kval in kernels:
    for cval in C:
        cv_scores =cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
        avg_scores[kval + '_' +str(cval)]=np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

this method also not convinent like running too many loops .
### sklern.model_selection.GridSearchCV for hyperparameter tuning
this api for overcome these problems

this api gooing to do exect same thing as above code

In [39]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False)

clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00110769, 0.00100121, 0.00148015, 0.00115151, 0.00104032,
        0.00081658]),
 'std_fit_time': array([7.66866235e-05, 1.46290236e-04, 4.23741131e-04, 2.15414816e-04,
        8.68386581e-05, 1.21687591e-04]),
 'mean_score_time': array([0.00060492, 0.00042777, 0.00053573, 0.0005528 , 0.00052795,
        0.00037527]),
 'std_score_time': array([1.98247502e-04, 1.19432241e-05, 7.31852616e-05, 8.55221235e-05,
        7.61557758e-05, 1.84819770e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [40]:
'''
It is hard to read as such so we convert it into pandas dataframe '''

'\nIt is hard to read as such so we convert it into pandas dataframe '

In [41]:
df=pd.DataFrame(clf.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001108,0.000077,0.000605,0.000198,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001001,0.000146,0.000428,0.000012,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001480,0.000424,0.000536,0.000073,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001152,0.000215,0.000553,0.000086,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001040,0.000087,0.000528,0.000076,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5


In [42]:
df[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [43]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_in_',
 'n_jobs',
 'n_splits

In [44]:
clf.best_score_

0.9800000000000001

In [45]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

## Disadvantage of GridsearchCV
- Its running cost is high for large datasets because it is using permutation and combination of the parameters.

#### To tackel this problem we have another sklearn library comesup with another class i.e. RandomizedSearchCV

## RandomizedSearchCV
- its cost is not too high because it will not try every permutation and conbination insted it takes random

In [48]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False
                      ,n_iter=2)#number of Combination we want to test

rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,1,rbf,0.980000


# Hyper parameter Tuning
## Choosing best model


In [49]:
# Checking for these three model
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

'''
Defining a dictonary with 
key: model name
values : dictonary to pass in the RandomizedCV with model and params i.e. parameters
'''

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [50]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}


# Conclusion
### Best model among these three is svm 